In [2]:
import numpy as np
from scipy.optimize import minimize

def classical_mds(D):
    """
    Классическая многомерная шкализация (Classical MDS) для полной матрицы расстояний.
    Возвращает координаты точек в 3D.
    
    :param D: np.ndarray, (N x N) — матрица расстояний
    :return: coords_mds (N x 3)
    """
    N = D.shape[0]
    # Матрица квадратов расстояний
    D2 = D**2

    # Центрирование
    J = np.eye(N) - np.ones((N, N)) / float(N)
    B = -0.5 * J @ D2 @ J

    # Собственные значения/вектора
    vals, vecs = np.linalg.eigh(B)  # eigh, так как B симметрична
    idx = np.argsort(vals)[::-1]
    vals = vals[idx]
    vecs = vecs[:, idx]


    lambdas = np.maximum(vals[:3], 0)
    L = np.diag(np.sqrt(lambdas))
    V = vecs[:, :3]

    coords_mds = V @ L
    
    return coords_mds

def pairwise_dist(c):
    """Считает попарные расстояния для массива координат c (N x 3)."""
    return np.linalg.norm(c[:, None, :] - c[None, :, :], axis=-1)

def distance_objective(x, D, w=None):
    """
    Функция ошибки для оптимизации:
    F = 1/2 * sum_{i<j} w_{ij} (||r_i - r_j|| - D[i,j])^2.
    x — это вектор длины 3N, мы его reshape -> (N, 3).
    w — веса (если None, считаем все равными 1).
    """
    N = D.shape[0]
    coords = x.reshape((N, 3))
    dist_current = pairwise_dist(coords)

    if w is None:
        err_matrix = (dist_current - D)**2
    else:
        err_matrix = w * (dist_current - D)**2

    np.fill_diagonal(err_matrix, 0.0)
    return 0.5 * np.sum(err_matrix)

def distance_jacobian(x, D, w=None):
    """
    Градиент функции distance_objective по x.
    
    Формула: dF/d r_i = sum_j [ (||r_i - r_j|| - D[i,j]) * ((r_i - r_j)/||r_i - r_j|| ) * w_{ij} ]
    """
    N = D.shape[0]
    coords = x.reshape((N, 3))
    dist_current = pairwise_dist(coords)

    grad = np.zeros_like(coords)

    if w is None:
        w_ij = np.ones((N, N), dtype=float)
    else:
        w_ij = w

    eps = 1e-12
    for i in range(N):
        diff = coords[i] - coords  # (N,3)
        r_ij = dist_current[i] + eps
        # factor = (||r_i - r_j|| - D[i,j]) / ||r_i - r_j|| * w_ij
        factor = (r_ij - D[i]) / r_ij * w_ij[i]
        grad[i] = np.sum(diff * factor[:, None], axis=0)
    
    grad *= 0.5

    return grad.ravel()

def refine_with_lbfgs(coords_init, D, w=None, max_iter=10000, tol=1e-6, verbose=True):
    """
    coords_init: (N x 3) — начальные координаты
    """
    x0 = coords_init.ravel()

    # Определим функцию-обёртку, чтобы вернуть и jac, и значение
    def func_and_grad(x):
        return distance_objective(x, D, w), distance_jacobian(x, D, w)

    res = minimize(func_and_grad, 
                   x0, 
                   method='L-BFGS-B', 
                   jac=True,
                   options={
                       'maxiter': max_iter,
                       'ftol': tol,
                       'gtol': 1e-8,
                       'disp': verbose
                   })
    coords_final = res.x.reshape((-1,3))
    return coords_final, res


if __name__ == "__main__":
    # Допустим «тетраэдр».
    true_coords = np.array([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.5, np.sqrt(3)/2, 0.0],
        [0.5, np.sqrt(3)/6, np.sqrt(6)/3]
    ], dtype=float)
    D = pairwise_dist(true_coords)

    # 1) Сделаем начальное приближение через classical MDS
    coords_mds = classical_mds(D)
    print("Начальная MDS-конфигурация:")
    print(coords_mds)

    # 2) Запустим L-BFGS-B оптимизацию
    coords_refined, res = refine_with_lbfgs(coords_mds, D, w=None, max_iter=5000)
    print("\nСтатус оптимизации:", res.message)
    print("Число итераций:", res.nit)
    print("Итоговая ошибка:", res.fun)

    print("\nИтоговые координаты (не выровненные):")
    print(coords_refined)

    # Посмотрим попарные расстояния
    D_final = pairwise_dist(coords_refined)
    print("\nРасстояния после оптимизации:")
    print(D_final)
    
    def kabsch_align(P, Q):
        # Сдвигаем центроиды в 0
        Pc = P - P.mean(axis=0)
        Qc = Q - Q.mean(axis=0)
        C = Pc.T @ Qc
        V, S, Wt = np.linalg.svd(C)
        d = (np.linalg.det(V) * np.linalg.det(Wt)) < 0.0
        if d:
            # Если отражение, разворачиваем знак в одной оси
            V[:, -1] = -V[:, -1]
        U = V @ Wt
        P_aligned = Pc @ U
        return P_aligned + Q.mean(axis=0)
    
    aligned = kabsch_align(coords_refined, true_coords)
    # Вычислим RMSD
    def rmsd(A, B):
        return np.sqrt(np.mean(np.sum((A - B)**2, axis=1)))

    print("\nВыровненная конфигурация:")
    print(aligned)

    r = rmsd(aligned, true_coords)
    print(f"RMSD c истинной (с учётом выравнивания): {r:.6f}")


Начальная MDS-конфигурация:
[[-0.52936867  0.30074964 -0.065715  ]
 [ 0.45025589  0.3188315  -0.26573692]
 [ 0.12295012 -0.06900464  0.59592082]
 [-0.04383733 -0.5505765  -0.26446891]]

Статус оптимизации: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Число итераций: 0
Итоговая ошибка: 4.190823558986625e-31

Итоговые координаты (не выровненные):
[[-0.52936867  0.30074964 -0.065715  ]
 [ 0.45025589  0.3188315  -0.26573692]
 [ 0.12295012 -0.06900464  0.59592082]
 [-0.04383733 -0.5505765  -0.26446891]]

Расстояния после оптимизации:
[[0. 1. 1. 1.]
 [1. 0. 1. 1.]
 [1. 1. 0. 1.]
 [1. 1. 1. 0.]]

Выровненная конфигурация:
[[5.55111512e-17 1.11022302e-16 2.77555756e-17]
 [1.00000000e+00 1.11022302e-16 5.55111512e-17]
 [5.00000000e-01 8.66025404e-01 2.22044605e-16]
 [5.00000000e-01 2.88675135e-01 8.16496581e-01]]
RMSD c истинной (с учётом выравнивания): 0.000000
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           12     M =           10

At X0         0

 This problem is unconstrained.
